# few-/zero-shot c GPT3

Скачаем и используем [ai-forever/rugpt3large_based_on_gpt2](https://huggingface.co/ai-forever/rugpt3large_based_on_gpt2/tree/main)

Идея классификации: используем loss модели для определения правильности построения фраз. Если фраза построена правильно, то у GPT не должно возникнуть затруднений в работе. Если фраза неправильная, то loss должен получиться выше.

Обучим гиперпараметр threshold (уровень разделения классов) по тренировочному набору данных, после этого проверим результаты на тестовом наборе данных.

Импорты

In [1]:
from sklearn.metrics import matthews_corrcoef
from torch.utils.data import TensorDataset
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import datetime
import numpy as np
import pandas as pd
import random
import time
import torch
import torch
import warnings
warnings.filterwarnings("ignore")

Параметры

In [2]:
PATH = r'H:\Инструменты\Windows\GPT or another LLM\ai-forever rugpt3large_based_on_gpt2 2020/'

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

## Необходимые определения

In [3]:
def calc_train_loss_threshold(model, tokenizer, df_train, n):
    positive_loss = []
    negative_loss = []

    for i in tqdm(range(min(len(df_train), n))):
        loss = inference(model, tokenizer, df_train.iloc[i].sentence)

        if df_train.iloc[i].acceptable == 1:
            positive_loss.append(loss)
        else:
            negative_loss.append(loss)

    # Здесь можно улучшить, если ввести доп.гиперпараметр, чтобы задавать: к какому классу ближе порог.
    # Сейчас усреднение делается ровно по среднему между объектами обоих классов.
    return (sum(negative_loss) / len(negative_loss) + sum(positive_loss) / len(positive_loss)) / 2.


def inference(model, tokenizer, phrase):
    with torch.no_grad():
        encodings = tokenizer(phrase, return_tensors='pt')
        input_ids = encodings.input_ids
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss

    return loss


def test(model, tokenizer, threshold, phrase, verbose = True):
    if verbose:
        print(phrase)

    loss = inference(model, tokenizer, phrase)

    if loss < threshold:
      return 1
    else:
      return 0


def get_test_predictions(model, tokenizer, threshold, df_test, n):
    predictions = []
    labels = []
    for i in tqdm(range(min(len(df_test), n))):
        acceptable_pred = test(model, tokenizer, threshold, df_test.iloc[i].sentence, verbose=False)
        predictions.append(acceptable_pred)
        labels.append(df_test.iloc[i].acceptable)

    return predictions, labels


def get_matthews_corrcoef_score(predictions, true_labels):
    # Calculate the MCC
    mcc = matthews_corrcoef(true_labels, predictions)
    return mcc

## Загрузим наборы данных

In [4]:
df_train = pd.read_csv('in_domain_train_subset.csv')
df_test = pd.read_csv('in_domain_test.csv')
print('Train dataset:\n', df_train.head())
print('Test dataset:\n', df_test.head())

Train dataset:
                                             sentence  acceptable
0                          О староверах я уже писал.           1
1  Христофоров, явившийся в ложу первым и одиноко...           0
2  Она так и сидела в гостинной, окруженная полто...           0
3  Он купит машину, если только не пропьет все де...           1
4                           Детям не было где спать.           1
Test dataset:
                                             sentence  acceptable
0                            Иван вчера не позвонил.           1
1  У многих туристов, кто посещают Кемер весной, ...           0
2  Лесные запахи набегали волнами; в них смешалос...           1
3  Вчера президент имел неофициальную беседу с ан...           1
4  Коллега так и не признал вину за катастрофу пе...           1


## Загрузим tokenizer и модель

In [5]:
print('Loading the tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(PATH)
print('Ok')

Loading the tokenizer...
Ok


In [6]:
model = GPT2LMHeadModel.from_pretrained(PATH)

### Обучим гиперпараметр threshold на тренировочном наборе данных

In [7]:
threshold = calc_train_loss_threshold(model, tokenizer, df_train, 7000)

100%|██████████| 6924/6924 [13:40<00:00,  8.44it/s] 


### Грубо проверим результат

In [11]:
print('threshold =', threshold)

phrases = ["Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.",
           "Вчера в два часа магазин закрыт.",
           'Кракадыл и друсья',
           'Крокодил Гена и его друзья.'
]
expecteds = [1, 0, 0, 1]

for phrase, expected in zip(phrases, expecteds):
    print('Expected:\t', expected)
    print('Predict:\t', test(model, tokenizer, threshold, phrase))
    print()

threshold = tensor(4.0431)
Expected:	 1
Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.
Predict:	 1

Expected:	 0
Вчера в два часа магазин закрыт.
Predict:	 0

Expected:	 0
Кракадыл и друсья
Predict:	 0

Expected:	 1
Крокодил Гена и его друзья.
Predict:	 1



Мы видим, что, в принципе, получается корректный результат.

### Проверим на тестовом наборе данных

In [9]:
predictions, true_labels = get_test_predictions(model, tokenizer, threshold, df_test, 1000)

100%|██████████| 983/983 [01:43<00:00,  9.49it/s]


In [12]:
mcc = get_matthews_corrcoef_score(predictions, true_labels)
print('Total MCC: %.3f' % mcc)

Total MCC: 0.072


Выводы:
- на грубых примерах подход работает хорошо: когда фразы явно неправильные — модель это определяет;
- на сложных тонких примерах тестового набора данных результаты получились низкие, на сложном dataset с тонкими дефектами построения фраз лучше результат с Bert.